In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
# from sklearn.preprocessing import StandardScaler
from joblib import dump
from pathlib import Path

In [2]:
url = 'https://raw.githubusercontent.com/jonyang6483/Project-4-Movie-Projections/main/Resources/clean_movies.csv'
df_movie = pd.read_csv(url)

In [3]:
df_movie = df_movie.drop(columns=['name', 'trixbudget'])

In [4]:
df_movie.columns


Index(['Unnamed: 0', 'rating', 'genre', 'year', 'released', 'director',
       'writer', 'star', 'country', 'budget', 'gross', 'company', 'runtime',
       'success'],
      dtype='object')

In [5]:
df_movie["director"].value_counts()

Woody Allen          33
Clint Eastwood       31
Steven Spielberg     27
Steven Soderbergh    23
Ridley Scott         23
                     ..
Takao Okawara         1
Mark Tarlov           1
Michael Corrente      1
Gary Halvorson        1
Hu Guan               1
Name: director, Length: 2063, dtype: int64

In [6]:
df = df_movie['director'].value_counts()
directors_to_replace = [index for index, x in df.items() if x < 2]

# Replace in dataframe
for i in directors_to_replace:
    df_movie['director'] = df_movie['director'].replace(i,"Other")

# Check to make sure binning was successful
df_movie['director'].value_counts()

Other                    1059
Woody Allen                33
Clint Eastwood             31
Steven Spielberg           27
Ron Howard                 23
                         ... 
Steve Boyum                 2
Kevin Rodney Sullivan       2
Troy Miller                 2
Frederik Du Chau            2
Adam Robitel                2
Name: director, Length: 1005, dtype: int64

In [7]:
df_movie_dummies = pd.get_dummies(df_movie)

In [8]:
df_movie_dummies

,Unnamed: 0,year,budget,gross,runtime,success,rating_Approved,rating_G,rating_NC-17,rating_Not Rated,...,company_Zentropa Entertainments,company_Zeta Entertainment,company_Zoetrope Studios,company_Zucker Brothers Productions,company_double A Films,company_erbp,company_i am OTHER,company_i5 Films,company_micro_scope,company_thefyzz
0,0,1980,19000000.0,46998772.0,146.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1980,4500000.0,58853106.0,104.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,1980,18000000.0,538375067.0,124.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1980,3500000.0,83453539.0,88.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,1980,6000000.0,39846344.0,98.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5416,7648,2020,90000000.0,426505244.0,124.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5417,7649,2020,85000000.0,319715683.0,99.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5418,7650,2020,175000000.0,245487753.0,101.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5419,7651,2020,135000000.0,111105497.0,100.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
movie_cat = df_movie_dummies.dtypes[df_movie_dummies.dtypes == "object"].index.tolist()

In [10]:
df_movie_dummies[movie_cat].nunique()

Series([], dtype: float64)

In [11]:
!pip install -U scikit-learn

enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_movie_dummies[movie_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(movie_cat)
encode_df.head()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


""
0
1
2
3
4


In [12]:
df_movie_dummies = df_movie_dummies.merge(encode_df,left_index=True, right_index=True)
df_movie_dummies = df_movie_dummies.drop(movie_cat,1)
df_movie_dummies.head()

<ipython-input-12-35883eee63dd>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_movie_dummies = df_movie_dummies.drop(movie_cat,1)


,Unnamed: 0,year,budget,gross,runtime,success,rating_Approved,rating_G,rating_NC-17,rating_Not Rated,...,company_Zentropa Entertainments,company_Zeta Entertainment,company_Zoetrope Studios,company_Zucker Brothers Productions,company_double A Films,company_erbp,company_i am OTHER,company_i5 Films,company_micro_scope,company_thefyzz
0,0,1980,19000000.0,46998772.0,146.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1980,4500000.0,58853106.0,104.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,1980,18000000.0,538375067.0,124.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1980,3500000.0,83453539.0,88.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,1980,6000000.0,39846344.0,98.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
y = df_movie_dummies["success"].values
X = df_movie_dummies.drop(columns=["success"],axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [14]:
X_train.shape

(4065, 7622)

In [15]:
X_test.shape

(1356, 7622)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Define the model - deep neural net
input_features = len(X_train[0])
hidden_nodes_layer1 =  2
hidden_nodes_layer2 = 4
hidden_nodes_layer3 = 1

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 15246     
                                                                 
 dense_1 (Dense)             (None, 4)                 12        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
 dense_3 (Dense)             (None, 1)                 2         
                                                                 
Total params: 15,265
Trainable params: 15,265
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
fit_model = nn.fit(X_train_scaled,y_train,epochs=20)

Epoch 1/20
128/128 [==============================] - 3s 10ms/step - loss: 0.6903 - accuracy: 0.5252
Epoch 2/20
128/128 [==============================] - 1s 11ms/step - loss: 0.6673 - accuracy: 0.5277
Epoch 3/20
128/128 [==============================] - 1s 8ms/step - loss: 0.6357 - accuracy: 0.6669
Epoch 4/20
128/128 [==============================] - 1s 8ms/step - loss: 0.6010 - accuracy: 0.7845
Epoch 5/20
128/128 [==============================] - 1s 8ms/step - loss: 0.5714 - accuracy: 0.8084
Epoch 6/20
128/128 [==============================] - 1s 7ms/step - loss: 0.5477 - accuracy: 0.8224
Epoch 7/20
128/128 [==============================] - 1s 7ms/step - loss: 0.5275 - accuracy: 0.8308
Epoch 8/20
128/128 [==============================] - 1s 7ms/step - loss: 0.5100 - accuracy: 0.8379
Epoch 9/20
128/128 [==============================] - 1s 8ms/step - loss: 0.4944 - accuracy: 0.8438
Epoch 10/20
128/128 [==============================] - 1s 8ms/step - loss: 0.4800 - accuracy: 0.84

In [20]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

43/43 - 0s - loss: 0.7632 - accuracy: 0.5752 - 489ms/epoch - 11ms/step
Loss: 0.7631865739822388, Accuracy: 0.5752212405204773
